In [1]:
pip install pyspark

     |████████████████████████████████| 212.4 MB 55 kB/s 
     |████████████████████████████████| 198 kB 51.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=b21161b2f0289b97399c54448d55233734dc92f5f02826bd47d992bef8fb0caa
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark=SparkSession.builder.appName('kmeans').getOrCreate()

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
file_path = '/content/drive/MyDrive/'
data=spark.read.csv(file_path +'movielens_ratings.csv',inferSchema=True,header=True)

In [7]:
data.head()

Row(movieId=2, rating=3.0, userId=0)

In [8]:
data.show()


+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [9]:
from pyspark.ml.recommendation import ALS  # alternating least square matrix facto

In [10]:
from pyspark.ml.evaluation import RegressionEvaluator

In [12]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [13]:
train_data,test_data=data.randomSplit([0.8,0.2])

In [14]:
als=ALS(maxIter=5,regParam=0.01,userCol='userId',itemCol='movieId',ratingCol='rating')

In [15]:
model=als.fit(train_data)

In [17]:
pred=model.transform(test_data)

In [18]:
pred.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     31|   1.0|    27| 0.8995853|
|     31|   2.0|    25|  2.023866|
|     85|   1.0|    26| 2.2087219|
|     85|   1.0|    12|0.92831904|
|     85|   3.0|     6| 3.8116286|
|     85|   1.0|     5| 2.8223612|
|     85|   1.0|     4| 2.2768185|
|     85|   3.0|    21|  3.651971|
|     85|   1.0|     2| 1.3086928|
|     65|   1.0|    22| 2.0189903|
|     65|   1.0|    16|-1.6608814|
|     65|   2.0|     5|  5.336091|
|     78|   1.0|    20| 1.0746927|
|     78|   1.0|    24|-0.0827659|
|     34|   1.0|     4|  1.091673|
|     81|   1.0|    22| 0.5196633|
|     81|   1.0|    19| 0.7010142|
|     76|   1.0|    19|0.11445776|
|     76|   5.0|    14| 0.6794031|
|     76|   3.0|    18| 3.6513834|
+-------+------+------+----------+
only showing top 20 rows



In [19]:
from pyspark.ml.evaluation import RegressionEvaluator

In [20]:
eval=RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction')

In [21]:
rmse=eval.evaluate(pred)

In [22]:
rmse

2.049636161866998